In [1]:
import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib

import pickle

#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale

#from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
#from gensim.models.wrappers import LdaMallet
#from gensim.corpora import Dictionary

#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random
from sklearn.externals import joblib


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel

warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp

import vectorizer
importlib.reload(vectorizer)
from vectorizer import  WeightedTfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import *
from sklearn.metrics import classification_report, confusion_matrix



# Reorganizamos el corpus

In [2]:
import joblib
corpus=joblib.load('../data/processed/reuters/corpus_reordered.final.pck')

In [3]:
corpus

,texto_normalizado,Target,Topic,prob_topic,outlier_iqr,new_target
0,asian exporter fear damage u.s.-japan rift mou...,0,0,0.006779,False,0
1,china daily say vermin eat 7 12 pct grain stoc...,0,0,0.011153,False,0
2,japan revise long term energy demand downwards...,0,0,0.009799,False,0
3,thai trade deficit widen quarter thailand trad...,0,0,0.009269,False,0
4,indonesia see cpo price rise sharply indonesia...,0,0,0.008703,False,0
...,...,...,...,...,...,...
10783,u.k money market shortage forecast revise bank...,0,0,0.015244,False,0
10784,knight ridder inc lt;krn > set quarterly qtly ...,0,1,0.986222,False,1
10785,technitrol inc lt;tnl > set quarterly qtly div...,0,1,0.986767,False,1
10786,nationwide cellular service inc lt;ncel > 4th ...,0,1,0.984818,False,1


In [4]:
corpusnormalizado = corpus.iloc[:,[0,5]]
corpusnormalizado.head()

,texto_normalizado,new_target
0,asian exporter fear damage u.s.-japan rift mou...,0
1,china daily say vermin eat 7 12 pct grain stoc...,0
2,japan revise long term energy demand downwards...,0
3,thai trade deficit widen quarter thailand trad...,0
4,indonesia see cpo price rise sharply indonesia...,0


In [5]:
corpussinoutliers = corpus.iloc[:,[0,2]]
corpussinoutliers.head()

,texto_normalizado,Topic
0,asian exporter fear damage u.s.-japan rift mou...,0
1,china daily say vermin eat 7 12 pct grain stoc...,0
2,japan revise long term energy demand downwards...,0
3,thai trade deficit widen quarter thailand trad...,0
4,indonesia see cpo price rise sharply indonesia...,0


In [6]:
topics0 = joblib.load('../objects/reuters/topics0')
topics1 = joblib.load('../objects/reuters/topics1')
feature_names = joblib.load('../objects/reuters/feature_names.final')

In [7]:
longitud = 100
vocab=topics1[-100:]
vocab2 = topics0[-200:]
contador = 0
for word in vocab2:
    if word not in vocab:
        contador = contador+1
        vocab.append(word)
        if contador == longitud:
            break

In [8]:
vocab

['55',
 'result',
 '52',
 'loan',
 'charge',
 '50',
 'credit',
 '42',
 '45',
 '1985',
 '40',
 '36',
 'june',
 '33',
 '51',
 '37',
 'period',
 'operation',
 'income',
 'extraordinary',
 '38',
 'seven',
 '34',
 'company',
 '32',
 '35',
 'ltd',
 'dlr',
 '29',
 'quarter',
 '26',
 '21',
 '23',
 'exclude',
 'end',
 '22',
 '24',
 '25',
 '19',
 '18',
 '27',
 'feb',
 'stock',
 'jan',
 'co',
 'tax',
 '14',
 '16',
 'payable',
 '17',
 '13',
 '20',
 'quarterly',
 'gain',
 '1987',
 'split',
 '11',
 '28',
 '3rd',
 '30',
 'set',
 'include',
 '12',
 '1986',
 'share',
 'march',
 '15',
 '10',
 '4th',
 'sale',
 'corp',
 'mths',
 'billion',
 '1st',
 'shrs',
 'avg',
 'year',
 'pay',
 '31',
 'prior',
 'oper',
 'inc',
 'qtly',
 'note',
 'april',
 'record',
 'dividend',
 'div',
 'lt',
 'dlrs',
 'profit',
 'rev',
 'qtr',
 'shr',
 'loss',
 '000',
 'net',
 'mln',
 'cts',
 'vs',
 'transaction',
 'debt',
 'follow',
 'contract',
 'house',
 'reduce',
 'approve',
 'agriculture',
 'average',
 'fund',
 'management',
 'g

In [10]:
wtfidf = WeightedTfidfVectorizer(vocabulary = vocab) #aquí ya hemos hecho una selección inicial de variables

In [26]:
newdtm = wtfidf.fit_transform(corpusnormalizado)

In [27]:
newdtm

<10788x200 sparse matrix of type '<class 'numpy.float32'>'
	with 173647 stored elements in Compressed Sparse Row format>

In [11]:
newdtmso = wtfidf.fit_transform(corpussinoutliers)

In [12]:
newdtmso

<10788x200 sparse matrix of type '<class 'numpy.float32'>'
	with 173647 stored elements in Compressed Sparse Row format>

In [29]:
joblib.dump(vocab,'../objects/reuters/vocab.dtm.final')
joblib.dump(newdtm,'../objects/reuters/newdtm.final')

In [15]:
joblib.dump(newdtmso,'../objects/reuters/newdtmsinoutliers.final')

['../objects/reuters/newdtmsinoutliers.final']

In [30]:
joblib.dump(corpusnormalizado,'../data/processed/reuters/corpus_normalizado.final')

['../data/processed/reuters/corpus_normalizado.final']

In [17]:
joblib.dump(corpussinoutliers,'../data/processed/reuters/corpus_normalizado_sin_outliers.final')

['../data/processed/reuters/corpus_normalizado_sin_outliers.final']